In [ ]:
%matplotlib inline
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from os import listdir
from os.path import join
from sklearn import metrics

# Creation of the data set

Real world data

Data source: Temp.csv

# One to one prediction

V1 column for 1 to 1 prediction. Shape: (1625,)



In [ ]:
d1=pd.read_csv("Temp.csv")
data=np.array(d1.V1)

# Data preparation

Arguments:

-time_wind: length of the sequence of the historical data;

-output: how many steps ahead prediction is going to be performed;

-nb_var: number of variables of the data set;

Output:

-x_batches: input for the training set. Size: (1518, 2, 1);

-y_batches: output for the training set. Size: (1518, 1);

-x_test: inputs for the test set; Size: (102, 2, 1);

-y_test: output for the test set; Size: (102, 1);

! Note, the size of the first dimension will vary depending on the size of the time_wind. The second dimension refers to the time_wind size, thisrd dimension: number of variables;

In [ ]:
def data_preparation(data,time_wind=2,output=1,nb_var=1,batch_size=1):
    data_train=data[:1520]
    x_train=[]
    for i in range((len(data_train)-1)-(time_wind-1)):
        x_train.append(data_train[i:(i+time_wind)])
    x_train=np.array(x_train)
    x_batches=(x_train[:(len(x_train)-(len(x_train)%batch_size))]).reshape(-1,time_wind,nb_var)
    y_train=data_train[time_wind:len(data_train)]
    y_batches=(y_train[:(len(y_train)-(len(y_train)%batch_size))]).reshape(-1,output)
    data_test=data[1521:]
    x_t=[]
    for i in range((len(data_test)-1)-(time_wind-1)):
        x_t.append(data_test[i:(i+time_wind)])
    x_t=np.array(x_t)
    x_test=(x_t[:(len(x_t)-(len(x_t)%batch_size))]).reshape(-1,time_wind,nb_var)
    y_t=data_test[time_wind:len(data_test)]
    y_test=(y_t[:(len(y_t)-(len(y_t)%batch_size))]).reshape(-1,output)
    return x_batches, y_batches, x_test, y_test

# TensorFlow computation graph

Arguments:

-epochs: how many times the network is going to be trained;

-time_wind: the length of the historical sequence;

-hidden: number of hidden layers;

-nb_var: amount of input variables;

-output: how many steps ahead prediction is going to be performed;

-lr: learning rate.

Outputs:

-mse_test: the mean squared error for the final prediction vector;

-y_pred: the vector of predictions.


In [ ]:
def run_graph(epochs,time_wind,hidden=10,nb_var=1,output=1,lr=0.01,r_s=1):

    from tensorflow.contrib import rnn
    tf.reset_default_graph()
    X=tf.placeholder(tf.float32,[None,time_wind,nb_var])
    Y=tf.placeholder(tf.float32,[None,output])
 
    tf.set_random_seed(r_s)
    x=tf.unstack(X,time_wind,1)
    
    basic_cell=tf.contrib.rnn.BasicRNNCell(num_units=hidden)
    rnn_output, states=rnn.static_rnn(basic_cell,x,dtype=tf.float32)

    stacked_rnn_output=tf.reshape(rnn_output[-1],[-1,hidden])
    stacked_outputs=tf.layers.dense(stacked_rnn_output,output)
    loss=tf.reduce_mean(tf.squared_difference(stacked_outputs, Y))
    optimizer=tf.train.AdamOptimizer(learning_rate=lr)
    training_op=optimizer.minimize(loss)
    init=tf.global_variables_initializer()
    with tf.Session() as sess:
        init.run()
        for ep in range(epochs):
            sess.run(training_op,feed_dict={X: x_batches, Y: y_batches})
            if ep % 100 == 0:
                mse=loss.eval(feed_dict={X: x_batches, Y: y_batches})
                print(ep,"\tMSE:",mse)
        y_pred=sess.run(stacked_outputs,feed_dict={X:x_test})
        mse_test=loss.eval(feed_dict={X:x_test,Y:y_test})
        print(mse)
        print(mse_test)
    sess.close() 
    return mse_test, y_pred

# Visualisation

In [ ]:
def visualisation():
    plt.plot(y_test,label='Actual')
    plt.plot(y_pred,label='Forecast')
    plt.legend(loc="upper left")
    return plt.show

# Experiments

## Naive models
### Average naive model

In [ ]:
x_batches, y_batches, x_test, y_test=data_preparation(data,1)
print(metrics.mean_squared_error(x_batches[:,0],np.full(len(x_batches),np.mean(x_batches))))

### Constant naive models

In [ ]:
x_t1=y_test[:102]
x_t=y_test[1:]
print(metrics.mean_squared_error(x_t,x_t1))

# Time_wind
The aim of the first  experiment is to define the optimal size of the time_wind

In [ ]:
mse_mean=[]
mse_var=[]
for j in range(1,9,1):
    mse=[]
    for i in range (10):
        x_batches, y_batches, x_test, y_test=data_preparation(data,j)
        mse_test,y_pred=run_graph(1000,j,hidden=1,lr=0.1,r_s=i)
        mse.append(mse_test)
    mse_mean.append(np.mean(mse))
    mse_var.append(np.var(mse))
    print("Mean of MSE for the model with time_wind={} equals{}".format(j, np.mean(mse)))
    print("Variance of MSE for the model with time_wind={} equals{}".format(j, np.var(mse)))
visualisation()

In [ ]:
plt.plot(mse_mean)

Summary: the highest accuracy was achieved with time_wind=2. Then MSE=8.106884. 
The accuracy of the basic model with time_wind=1 is equal to 15.629408, what is better the the results of the naive model.

# Hidden layers
The aim of the second experiment is to discover wether the increment of the number of hidden layers will improve the accuracy

In [ ]:
mse_mean=[]
mse_var=[]
for j in range(1,11,1):
    mse=[]
    for i in range (10):
        x_batches, y_batches, x_test, y_test=data_preparation(data,2)
        mse_test,y_pred=run_graph(1000,2,hidden=j,lr=0.1,r_s=i)
        mse.append(mse_test)
    mse_mean.append(np.mean(mse))
    mse_var.append(np.var(mse))
    print("Mean of MSE for the model with hidden={} equals{}".format(j, np.mean(mse)))
    print("Variance of MSE for the model with hidden={} equals{}".format(j, np.var(mse)))
visualisation()
    

In [ ]:
plt.plot(mse_mean)

Summary: The lowest MSE is achieved with hidden=6. MSE=5.53754. 
The structure of the data is too complex to hadle it with the smaller size of hidden states, however, the bigger amount of hiddent units cause overfitting.

# Optimisation of the training time

## Learning rate: 0.1

In [ ]:
mse_mean=[]
mse_var=[]
for j in range(1000,11000,1000):
    mse=[]
    for i in range (10):
        x_batches, y_batches, x_test, y_test=data_preparation(data,2)
        mse_test,y_pred=run_graph(j,2,hidden=6,lr=0.1,r_s=i)
        mse.append(mse_test)
    mse_mean.append(np.mean(mse))
    mse_var.append(np.var(mse))
    print("Mean of MSE for the model with epochs={} equals {}".format(j, np.mean(mse)))
    print("Variance of MSE for the model with epochs={} equals {}".format(j, np.var(mse)))
visualisation()

In [ ]:
plt.plot(mse_mean)

Summary:
With the learning rate equal to 0.01, increasing of the training time does not improve the quality of predictions.

## Learning rate: 0.01


In [ ]:
mse_mean=[]
mse_var=[]
for j in range(2000,11000,1000):
    mse=[]
    for i in range (10):
        x_batches, y_batches, x_test, y_test=data_preparation(data,2)
        mse_test,y_pred=run_graph(j,2,hidden=6,lr=0.01,r_s=i)
        mse.append(mse_test)
    mse_mean.append(np.mean(mse))
    mse_var.append(np.var(mse))
    print("Mean of MSE for the model with epochs={} equals {}".format(j, np.mean(mse)))
    print("Variance of MSE for the model with epochs={} equals {}".format(j, np.var(mse)))
visualisation()

In [ ]:
plt.plot(mse_mean)

Summary:  With learning rate equal to 0.01, the highest accuracy was achieved with number of training epochs equal to 3000. Then, MSE=5.66024

## Learning rate: 0.001

In [ ]:
mse_mean=[]
mse_var=[]
for j in range(4000,17000,1000):
    mse=[]
    for i in range (10):
        x_batches, y_batches, x_test, y_test=data_preparation(data,2)
        mse_test,y_pred=run_graph(j,2,hidden=6,lr=0.001,r_s=i)
        mse.append(mse_test)
    mse_mean.append(np.mean(mse))
    mse_var.append(np.var(mse))
    print("Mean of MSE for the model with epochs={} equals {}".format(j, np.mean(mse)))
    print("Variance of MSE for the model with epochs={} equals {}".format(j, np.var(mse)))
visualisation()

In [ ]:
plt.plot(mse_mean)

Summary: with the learnig rate equal to 0.001, the lowes MSE was achieved with number of epoch equal to 9000. MSE=5.69389

 # Many to one prediction

V1, V2 columns for 2 to 1 prediction. Shape: (2,1625)

In [ ]:
data2=np.array([d1.V1,d1.V2])

# Data preparation

In [ ]:
def data_preparation2(data,time_wind=2,output=1,nb_var=2,batch_size=1):
    data_train=data[:1520]
    x_train=[]
    for i in range((len(data_train)-1)-(time_wind-1)):
        x_train.append(data_train[i:(i+time_wind)])
    x_train=np.array(x_train)
    x_batches=(x_train[:(len(x_train)-(len(x_train)%batch_size))]).reshape(-1,time_wind,nb_var)
    y_train=data_train[time_wind:len(data_train)]
    y_batches=(y_train[:,0][:(len(y_train)-(len(y_train)%batch_size))]).reshape(-1,output)
    data_test=data[1521:]
    x_t=[]
    for i in range((len(data_test)-1)-(time_wind-1)):
        x_t.append(data_test[i:(i+time_wind)])
    x_t=np.array(x_t)
    x_test=(x_t[:(len(x_t)-(len(x_t)%batch_size))]).reshape(-1,time_wind,nb_var)
    y_t=data_test[time_wind:len(data_test)]
    y_test=(y_t[:,0][:(len(y_t)-(len(y_t)%batch_size))]).reshape(-1,output)
    return x_batches, y_batches, x_test, y_test

# Time_wind

In [ ]:
mse_mean=[]
mse_var=[]
for j in range(1,9,1):
    mse=[]
    for i in range (10):
        x_batches, y_batches, x_test, y_test=data_preparation2(data2.T,j,nb_var=2)
        mse_test,y_pred=run_graph(1000,j,hidden=1,lr=0.1,r_s=i,nb_var=2)
        mse.append(mse_test)
    mse_mean.append(np.mean(mse))
    mse_var.append(np.var(mse))
    print("Mean of MSE for the model with time_wind={} equals{}".format(j, np.mean(mse)))
    print("Variance of MSE for the model with time_wind={} equals{}".format(j, np.var(mse)))
visualisation()

In [ ]:
plt.plot(mse_mean)

Summary: The lowest MSE was achieved with the time window of size 2 (time_wind=2). MSE=6.4604

# Hidden layers

In [ ]:
mse_mean=[]
mse_var=[]
for j in range(1,11,1):
    mse=[]
    for i in range (10):
        x_batches, y_batches, x_test, y_test=data_preparation2(data2.T,2,nb_var=2)
        mse_test,y_pred=run_graph(1000,2,hidden=j,lr=0.1,r_s=i,nb_var=2)
        mse.append(mse_test)
    mse_mean.append(np.mean(mse))
    mse_var.append(np.var(mse))
    print("Mean of MSE for the model with hidden={} equals{}".format(j, np.mean(mse)))
    print("Variance of MSE for the model with hidden={} equals{}".format(j, np.var(mse)))
visualisation()

In [ ]:
plt.plot(mse_mean)

Summary: the highest accuracy was achieved with the hidden state size equal to hidden=5. MSE=4.64192

# Optimisation of the training time

## Learning rate: 0.1

In [ ]:
mse_mean=[]
mse_var=[]
for j in range(1000,11000,1000):
    mse=[]
    for i in range (10):
        x_batches, y_batches, x_test, y_test=data_preparation2(data2.T,2,nb_var=2)
        mse_test,y_pred=run_graph(j,2,hidden=5,lr=0.1,r_s=i,nb_var=2)
        mse.append(mse_test)
    mse_mean.append(np.mean(mse))
    mse_var.append(np.var(mse))
    print("Mean of MSE for the model with epochs={} equals {}".format(j, np.mean(mse)))
    print("Variance of MSE for the model with epochs={} equals {}".format(j, np.var(mse)))
visualisation()

In [ ]:
plt.plot(mse_mean)

Summary: with the learning rate equal to 0.1, the increment of the training time does not inprove the accuracy of the prediction.

## Learning rate: 0.01

In [ ]:
mse_mean=[]
mse_var=[]
for j in range(1000,11000,1000):
    mse=[]
    for i in range (10):
        x_batches, y_batches, x_test, y_test=data_preparation2(data2.T,2,nb_var=2)
        mse_test,y_pred=run_graph(j,2,hidden=5,lr=0.01,r_s=i,nb_var=2)
        mse.append(mse_test)
    mse_mean.append(np.mean(mse))
    mse_var.append(np.var(mse))
    print("Mean of MSE for the model with epochs={} equals {}".format(j, np.mean(mse)))
    print("Variance of MSE for the model with epochs={} equals {}".format(j, np.var(mse)))
visualisation()

In [ ]:
plt.plot(mse_mean)

Summary: With the learning rate equal to 0.01 the highest accuracy was achieved with number of epochs=4000. MSE=4.37898

## Learning rate: 0.001

In [ ]:
mse_mean=[]
mse_var=[]
for j in range(4000,15000,1000):
    mse=[]
    for i in range (10):
        x_batches, y_batches, x_test, y_test=data_preparation2(data2.T,2,nb_var=2)
        mse_test,y_pred=run_graph(j,2,hidden=5,lr=0.001,r_s=i,nb_var=2)
        mse.append(mse_test)
    mse_mean.append(np.mean(mse))
    mse_var.append(np.var(mse))
    print("Mean of MSE for the model with epochs={} equals {}".format(j, np.mean(mse)))
    print("Variance of MSE for the model with epochs={} equals {}".format(j, np.var(mse)))
visualisation()

In [ ]:
plt.plot(mse_mean)

Summary: with the learning rate 0.001, the highest accuracy is achieved with number of epochs=10000. MSE=4.38957